In [1]:
# this page contains live examples of casa usage from the following installation
!pip install casaconfig  > /dev/null

# External Data

Each CASA distribution requires a runtime configuration and minimal repository of binary data for CASA to function properly. This is contained in a casarundata tar file which must be installed before CASA can be used. The data repository includes Measures Tables that deal with the Earth Orientation Parameters (EOPs), ephemeris data, antenna configurations, beam models, and calibration corrections. The **casaconfig module** provides functions along with a default configuration file that properly sets up and maintains the data repository contents.

Once an initial data respository is installed **CASA maintains this system automatically**, placing small daily updates of measures data in the *measurespath* location. Occasional updates of the other parts of the data repository are updated as they become available. The default *measurespath* is ~/.casa/data.  Site installations of CASA will typically set *measurespath* to a shared location that the site administrators maintain (automatic updates are then turned off by the site administrators). Users may modify or disable this functionality, including adding their own, personally maintained, *measurespath* and controlling which data is automatically updated (all of casarundata or just the measures data) by setting configuration values.

CASA now uses a hierarchy of configuration files. The default configuration values are first set by **config.py** in casaconfig. If a **casasiteconfig.py** is found then that is used next. Finally if the user has a personal **config.py** in ~/.casa then that is used. The full configuration is evaluated by importing **config** from casaconfig.

It is not necessary to set all of the configuration parameters in each of the optional configuration files. The configuration files can consist of any executable python. They should not depend on CASA modules since the configuration must be set before a CASA module can be used.

Monolithic CASA (the casashell module) has command line options that can also be used to change the configuration values. The command line option values always take precedence over any configuration value set in a config file. The location of the user's configuration file can be set on the commandline. The commnad line can be used to skip both the site and user's configuration files.

The following figure depicts a high level view of the external data management system, including the casarundata from the CASA site, modular pip package and installation, monolothic tarball and installation, and ASTRON ftp server.

![casaconfigsystem](https://github.com/casangi/casadocs/blob/CAS-13116/docs/notebooks/media/casaconfig_system.png?raw=1)


The following sections illustrate how to manually manipulate data contents. By default, CASA will generally handle this automatically after the initial installation unless the user overrides the settings with their own ~/casa/config.py file.



## Locating the Data Directory

The *measurespath* configuration value is used to find the data directory. The default location is ~/.casa/data. Site installations will typically set that to a shared location in a site configuration file found either at the value of a CASASITECONFIG environment variable if set or at /opt/casa/casasiteconfig.py, /locat/casa/casasiteconfig.py or a casasiteconfig.py found in the python path (typically in the site-packages directory). An example casasiteconfig.py is provided in casaconfig/private/casasiteconfig_example.py.

The default *measurespath* is ~/.casa/data as seen below. Unless you've previously installed data there this location will not exist.

In [2]:
# get the configuration and show the value of measurespath
from casaconfig import config
print(config.measurespath)

/root/.casa/data


In [3]:
# see what's in it
!ls $config.measurespath

ls: cannot access '/root/.casa/data': No such file or directory


In [4]:
# the default auto update config values are True (enabled)
print(config.data_auto_update)
print(config.measures_auto_update)

True
True


## Populating the Data Directory Automatically

It is necessary to first create and populate *measurespath*. The CASA modules will not create *measurespath* to avoid surprises.

When CASA starts (when the casatools module is being initialized during import) if *measurespath* exists but is empty, and it is owned by the user, and the auto update values are True then the casaconfig modules will be used to populate *measurespath* automatically.

So for most users the following is sufficient to initially populate *measurespath* (from the OS prompt):


```
    mkdir ~/.casa/data
    <path_to_casa_bin>/casa
```

The casaconfig module methods will then download the casarundata tarball, installed it in ~/.casa/data, download the measures tarball from ASTRON, and update the measures tables that were just installed from the casarundata tarball. Regular CASA use can then continue in the casa session just started. Subsequent uses of casa will check for measures and data updates and install them if new versions are available. The auto update steps only check for updates once per day. The measures tables are updated daily and the full casarundata tarball is rarely updated (typically at each new CASA release and infrequently between releases as necessary).

The size of the installed data is about 831M. That size may be larger than user want to keep in a directory under their home directory. In that case, users should create a personal config file in ~/.casa/config.py (if they don't already have one) and set measurespath to point at that location. As with the default location, if *measurespath* exists but is empty and owned by the user, starting CASA or importing casatools will populate that location when auto updates are enabled.

```
# an example config.py setting measurespath
measurespath = "/path/to/measurespath"
```

Subsequent installations of CASA can continue to use the same measurespath and automatic updates will continue to install the data as it becomes available.

Site installations where multiple users use the same CASA and the same (shared) meaurespath need to maintain meaurespath for those users. The site config file should turn off auto updates and set measurespath. The command line casaconfigs options can be used to maintain measurespath. Legacy locations of the CASA data can continue to be used as described later (casaconfig will not update those types of data installations).

The *measurespath* location must be populated manually. This example uses **pull_data** to first install the most recent casarundata tarball and then in a later cell **measures_update** is used to make sure that the contents are up to date. That step would be done automatically the next time that CASA is used if automatic updates are turned on in the configuration and the user owns *measurespath*. The **update_all** function can be used to install and update the data with a single function call.

If this line is rerun the data is not re-installed because casaconfig sees that the expected version has already been installed.


In [ ]:
# populate measurespath, which is created as needed provided that the user has persmissions to do that
from casaconfig import pull_data
pull_data()
!ls $config.measurespath

pull_data using version casarundata-0.0.6.tar.gz, acquiring the lock ... 
downloading casarundata contents to /root/.casa/data (333M) ... done
casarundata installed casarundata-0.0.6.tar.gz at /root/.casa/data
alma  catalogs	data_update.lock  demo	ephemerides  geodetic  gui  nrao  readme.txt


This example illustrates how to populate a custom location. The *measurespath* config value wouild need to be set to use that location in CASA.

In [ ]:
# populate some custom location with the data folder contents
pull_data('./casadata')

!ls ./casadata

pull_data using version casarundata-0.0.6.tar.gz, acquiring the lock ... 
downloading casarundata contents to ./casadata (333M) ... done
casarundata installed casarundata-0.0.6.tar.gz at ./casadata
alma  catalogs	data_update.lock  demo	ephemerides  geodetic  gui  nrao  readme.txt


<div class="alert alert-warning">
**WARNING**: If you are using python-casacore directly (outside of CASA), you will need to set your .casarc file to point to wherever you installed casaconfig and/or populated a data folder. CASA does not use python-casacore so this step is not necessary when using CASA.
</div>

In [ ]:
# tell casacore where to find casaconfig
from casaconfig import set_casacore_path
set_casacore_path(config.measurespath)
!more ~/.casarc

writing /root/.casarc...
measures.directory: /root/.casa/data


## Updating the Data Directory

Most of the data tables (such as beam models, antenna and Jy/K correction tables, and the antenna configuration files for the CASA simulator) are versioned by CASA release and seldom change. However, the **Casacore Measures** tables (ie *geodetic* subdirectory) must be updated frequently after release.

It is important to know what version/date of Measures data is currently populated. This can be done by examining the *readme.txt* file in the geodetic subdirectory.

In [ ]:
!cat ~/.casa/data/geodetic/readme.txt

# measures data populated by casaconfig
version : WSRT_Measures_20231029-160001.ztar
date : 2023-11-01

The ```measures_update()``` function is used to download new measures data from the originating source. By default, this function will retrieve the latest data.  If you already have the latest data, then nothing will happen.

In [ ]:
from casaconfig import measures_update
measures_update()
!cat ~/.casa/data/geodetic/readme.txt

measures_update measures need to be updated, acquiring the lock ... 
measures_update connecting to ftp.astron.nl ...
measures_update downloading WSRT_Measures_20231101-160001.ztar from ASTRON server to /root/.casa/data ...
measures_update updated measures data at /root/.casa/data
# measures data populated by casaconfig
version : WSRT_Measures_20231101-160001.ztar
date : 2023-11-02

Specific versions of past measures data can be retrieved as well. This may be important if trying to exactly replicate the conditions of a particular data reduction run in CASA.  Generally though the measures data is appended with time, so past and current versions should have the same values at the same points in time (see later section of casacore measures data contents).

In [ ]:
# see if something newer is available
from casaconfig import measures_available
versions = measures_available()
print(versions[-3:])

['WSRT_Measures_20231030-160001.ztar', 'WSRT_Measures_20231031-160001.ztar', 'WSRT_Measures_20231101-160001.ztar']


In [ ]:
# retrieve a version from a while back
measures_update(version=versions[-10], force=True)
!cat ~/.casa/data/geodetic/readme.txt

measures_update measures need to be updated, acquiring the lock ... 
meaures_update a measures update has been requested by the force argument
measures_update connecting to ftp.astron.nl ...
measures_update downloading WSRT_Measures_20231023-160001.ztar from ASTRON server to /root/.casa/data ...
measures_update updated measures data at /root/.casa/data
# measures data populated by casaconfig
version : WSRT_Measures_20231023-160001.ztar
date : 2023-11-02

**Note: measures_update()** requires that the expected readme.txt file exists at the location being updated. This helps to protect against updating a location that is being maintained outside of the casaconfig methods.

**Note:** when auto updates are enabled (*measures_auto_update* or *data_auto_update* are True) then thsi step happens each time that CASA is started (during initialization of the casatools module). Auto updates require that the user owns the location being updated in addition to the requirement that the expected readme.txt file exists. This helps to protect against updating a location that is shared by multiple users (e.g. a site *measurespath*). If today's date is the same as the date in the readme.txt file or the version is the most recent available version then no  update will be done.

## Data Locations

There are two configuration values related to where CASA finds the external data described here: *measurespath* and *datapath*.

The *datapath* value is a list of locations to be searched, in order, for the desired data. That value is used by the **resolve()** method of the *utils* tool (*ctsys* in a running CASA environment). The *datapath* value can be changed after CASA starts and the new value will be used in subsequent searches.

The *measurespath* value is used when the casatools module starts to find the location of the IERS tables. Those values are read at that point and used throughout the CASA session. Changing *measurespath* after casatools has started will **NOT** cause the IERS tables at the new location to be used. Updating the IERS tables after casatools has startetd will not change the IERS values used by that session of CASA. CASA must be restarted to see any changes in the IERS tables.

The default *datapath* has *measurespath* as the first and only element in the list. CASA recommends that measures data (**measures_update()**) be kept at the same location as casarundata (**pull_data()** and **data_update()**) and that that location be the first element of *datapath*. Other arrangements may work but are not tested by CASA and may be confusing.

Except for the IERS data as described previously, data used by CASA is read as needed. Changes to non-IERS data after CASA starts may be seen during that CASA session (depending on whether other tools have already read those files). Updates for casarundatea are provided infrequently.

Shared data used by one or more installations maintained at a site are typically updated at a time not controlled by a CASA user. User's concerned about that may choose to install their own copy of the data using the casaconfig module methods as descrbed here. They should set *measurespath* in their config.py to point to a location of their choosing. Typically they will turn on automatic updates so that their installed data is updated as needed when CASA starts but they may choose to not use automatic updates if they may run multiple CASA sessions at the same time and they want to ensure that the same data is used for a long-running session, for example.

## Site Installations

Monolithic CASA (distributed in a tarball) is configured for use by a site where one or more CASA installations are shared by multiple users.

The location of *measurespath* is set in the **casasiteconfig.py**. The CASA distribution comes with a **casasiteconfig.py** that must be edited to set *measurespath* (the default value is None). The site maintainer can choose any location. The **casasiteconfig.py** can be found in the site-packages/casaconfig directory (e.g. <casa-install-path>/lib/py/lib/python3.8/site-packages/casaconfig/).

This is the default **casasiteconfig.p**.
```
# The default site config file. This may be placed at any location in the PYTHON
PATH used by CASA

# This file should be edited to set measurespath as appropriate

# Set this to point to the location where the site maintained casarundata can be
 found
# by default datapath will include measurespath

measurespath = None

# turn off all auto updates of data

measures_auto_update = False
data_auto_update = False
```
Once *measurespath* has been set in **casasiteconfig.py** the location must be populated to contain the expected data. The same *measurespath* can be used by multiple installations. Note that auto updates are False here so that user's don't attempt to update the site *measurespath* when they start CASA. The auto update mechanism also checks that the user is the owner of *measurespath* before any attempt to update it is made. An error is printed and CASA continues starting if *measurespath* can not be updated when either auto update is True.

**Note:** Sites with older installations that already have CASA data shared across multiple sites can use the same location and update methods that they have been using. If this legacy update mechanism is used be aware that CASA is no longer distributed with a **data** or **__data__** directory internal to the distribution so replacing that with a link to the site location is no longer necessary and doing that will not allow that installation of CASA to find the data. The *measurespath* **must** indicate where the CASA data can be found. Be aware that the **update-data** script accompanying CASA now assumes that the data are being maintained using **casaaconfig** methods and not the legacy procedures. The two methods are not compatible and using the new **update-data** script will result in errors and that data will not be updated. Legacy **update-data** scripts and site-maintained procedures will continue to work for some time but are likely to become inoperable at some future time. CASA recommends swithcing to the new update methods described here. Data installed for use with CASA using the new update methods can be used with older CASA installations.

The **casaconfig** module has command-line options that are useful for maintaining casarundata once *measurespath* has been set (this is also true for user's maintaining their own *measurespath*).

At the shell prompt, use the python3 that accompanies the "casa" script.

```
$ <path_to_casa_installation>/bin/python3 -m casaconfig --help
sage: __main__.py [-h] [--configfile CONFIGFILE] [--noconfig] [--nositeconfig] [--measurespath MEASURESPATH] [--pull-data] [--data-update] [--measures-update] [--update-all] [--reference-testing]
                   [--current-data]

optional arguments:
  -h, --help            show this help message and exit
  --configfile CONFIGFILE
                        location of the user configuration file
  --noconfig            do not load user configuration file
  --nositeconfig        do not load site configuration file
  --measurespath MEASURESPATH
                        location of casarundata
  --pull-data           invoke pull_data() to populate measurespath with latest casarundata
  --data-update         invoke data_update() to update measurespath to the latest casarundata
  --measures-update     invoke measures_update() to update measurespath to the latest measures data
  --update-all          invoke update_all() to populate (update) measurespath with the latest casarundata and measures data
  --reference-testing   set measurespath to the casarundata when this version was produced, used for testing purposes
  --current-data        print out a summary of the current casarundata and measures data installed in measurespath and then exit

```
The *--update-all* option can be used to populate *measurespath* and keep it up to date with both the latest casarundata and the daily measures updates. Other options exist to do specific updates (casarundata or measures) separately. Note that casarundata always includes the measures data from the date when that casarundata version was produced. That will typically be older than the most recent measures data and a casarundata update is typically immediately followed by a meaures update, which is what *--update-all* does.

If *measurespath* does not exist or is empty then *--update-all* and *--pull-data** will create it if necessary before populating it.

If *measurespath* exists and contains something but the expected readme.txt files are not found then nothing will be done to the contents at that location and an error message will be shown describing that problem. This is most likely due to *measurespath* pointing a legacy casa data location. The casaconfig module requires that the readme.txt files are found and can be read as expected before it makes any changes to the contents at *measurespath*.

Use *--update-all* to populate *measurespath* and do that daily to keep it up to date. The **update-data** script can be used as before to do the same thing, but that typically requires a running CASA session in order to easily locate that script and that requires that *measurespath* already be populated.

```
$ python3 -m casaconfig --update-all
data_update then measures_update using path=<your_measurespath>
pull_data using version casarundata-0.0.6.tar.gz, acquiring the lock ...
downloading casarundata contents to <your_measurespath> (333M) ... done
casarundata installed casarundata-0.0.6.tar.gz at <your_measurespath>
data_update current casarundata detected in <your_measurespath>, using version casarundata-0.0.6.tar.gz
measures_update measures need to be updated, acquiring the lock ...
measures_update connecting to ftp.astron.nl ...
measures_update downloading WSRT_Measures_20231101-160001.ztar from ASTRON server to <your_measurespath> ...
measures_update updated measures data at <your_measurespath>
```

The *--current-data* option is used to show what *measurespath* current contains.

```
$ python3 -m casaconfig --current-data
casarundata version casarundata-0.0.6.tar.gz installed on 2023-11-02
measures version WSRT_Measures_20231101-160001.ztar installed on 2023-11-02
```

## Reference Testing

Testing often requires that the casarundata be set to a known version. Monolithic tarball is packaged with information that casaconfig can use to set the casarundata to the version at the time that that release was set. The **casaconfig** command line *--reference-testing* argument can be used to set the data at *measurespath* to that version. It is intended for internal testing only. This sets the data at *measurespath* to what it would have been had CASA been packaged with data at the time that version of CASA was released. This includes the measures data at that date (i.e. the measures data will be out of date, which can be useful for testing).

```
$ python3 -m casaconfig --reference-testing
```

## Data Directory Contents

### Casacore Measures

The casacore Measures tables are needed to perform accurate conversions of reference frames.  Casacore infrastructure includes classes to handle physical quantities with a reference frame, so-called *Measures*. Each type of Measure has its own distinct class in casacore which is derived from the Measure base class. One of the main functionalilties provided by casacore w.r.t. Measures, is the conversion of Measures from one reference frame to another using the MeasConvert classes.

Many of the spectral, spatial, and time reference frames are time-dependent and require the knowledge of the outcome of ongoing monitoring measurements of properties of the Earth and astronomical objects by certain service observatories. This data is tabulated in a number of tables (*Measures Tables*) which are stored in the casadata repository in the subdirectory ```geodetic```. A snapshot of this repository is included in each tarball distribution of CASA and in the casadata module for CASA6+.

Measures tables are updated daily based on the refinement of the geodetic information from the relevant services like the International Earth Rotation and Reference Systems Service (IERS). Strictly speaking, the Measures tables are part of the casacore infrastructure which is developed by NRAO, ESO, NAOJ, CSIRO, and ASTRON. In order to keep the repository consistent between the partners, the Measures tables are initially created at a single institution (ASTRON) and then copied into the NRAO casadata repository from which all CASA users can retrieve them. As of March 2020, the update of the NRAO CASA copy of the Measures tables in ```geodetic``` and the planetary ephemerides in directory ```ephemerides``` takes place every day between 18 h UTC and 19 h UTC via two redundant servers at ESO (Garching).

**CASA releases need to be updated with recent Measures tables** (see [above](#Updating-the-Data-Directory)).  For observatory use, the update period should not be longer than weekly in order to have the EOPs up-to-date for upcoming observations. The shortest reasonable update interval is daily. For offline data analysis use, the update period should not be longer than monthly. Weekly update is recommended.

Legacy installations processing old data do not have to be updated because the relevant contents of the Measures Tables is not changing any more for the more distant past.

The following list describes the individual Tables in subdirectory ```geodetic```:

* **IERSeop2000:**    The IERS EOP2000C04_05 Earth Orientation Parameters
using the precession/nutation model \"IAU2000\" (files eopc04_IAU2000.xx)
* **IERSeop97**:    The IERS EOPC04_05 Earth Orientation Parameters using the precession/nutation model \"IAU 1980\" (files eopc04.xx)
* **IERSpredict:** IERS Earth Orientation Data predicted from NEOS (from file <ftp://ftp.iers.org/products/eop/rapid/daily/finals.daily>)
* **IGRF:**    International Geomagnetic Reference Field Schmidt [semi-normalised spherical harmonic coefficients](https://www.ngdc.noaa.gov/IAGA/vmod/coeffs/). (Note that this still uses IGRF12. An update to IGRF13 is underway.)
* IMF (not a Measures Table proper, access not integreated in Measures framework):    Historical interplanetary magnetic field data until MJD 52618 (December 2002).
* KpApF107 (not a Measures Table proper, access not integreated in Measures framework): Historical geomagnetic and solar activity indices until MJD 54921 (April 2009)
* **Observatories:** Table of the official locations of radio observatories. Maintained by CASA.
* SCHED_locations (not a Measures Table proper, access not integreated in Measures framework):    VLBI station locations
* **TAI_UTC:** TAI_UTC difference (i.e. leap second information) obtained from USNO


Measures Tables in the directory ```ephemerides```:

* **DE200**:    Historical JPL Planetary ephemeris DE200 used for Astronomical Almanach from 1984 to 2003 (from <ftp://ssd.jpl.nasa.gov/pub/eph/planets/ascii/de200>)
* **DE405**:    JPL Planetary ephemeris DE405; includes nutations and librations; referred to the ICRS (from <ftp://ssd.jpl.nasa.gov/pub/eph/planets/ascii/de405>)



### Ephemeris Data

The ephemeris tables hold a selection of the solar system objects from JPL-Horizons database.  The data tables are generated from the JPL Horizons system\'s on-line solar system data and ephemeris computation service ([https://ssd.jpl.nasa.gov/?horizons )](https://ssd.jpl.nasa.gov/?horizons "JPL Horizons"). These are primarily used to determine flux models for the solar system objects used in the **setjy** task.  These tables are stored as CASA tables in the casadata repository under ```ephemerides/JPL-Horizons```. The current ephemeris tables cover ephemerides until December 31, 2030 for those objects officially supported in **setjy**.

Available objects, which include major planets, satellites, and asteroids, are: Mercury, **Venus**, **Mars**, **Jupiter**, Saturn, **Uranus**, **Neptune**, Pluto, **Io**, **Europa**, **Ganymede**, **Callisto**, **Titan**, **Ceres**, **Vesta**, **Pallas**, **Juno**, **Lutetia**, Sun and Moon (the objects in **bold** are those supported in \'Butler-JPL-Horizons 2012\' standard in **setjy**.).

The format of the table name of these tables is *objectname*\_*startMJD*\_*endMJD*\_J2000.tab These tables required by **setjy** task are included in the data directory in the CASA distribution. The available tables can be listed by the following commands:


```
#In CASA6

CASA <1>: import glob

CASA <2>: jpldatapath = ctsys.resolve('ephemerides/JPL-Horizons') + "/*J2000.tab"

CASA <3>: glob.glob(jpldatapath)
```

The following data are retrieved from the JPL-Horizons system (the nubmer in the parentheses indicates the column number listed in the JPL-Horizons system). One should refer https://ssd.jpl.nasa.gov/?horizons_doc for the detailed descreption of each of these quantities.

Quantities                   | column no. | Unit/format            | Descrition                                                                                                                 | column label
---                          | ---        | ---                    | ---                                                                                                                        | ---
Date                         | n.a.       | YYYY-MM-DD             | HH:MM                                                                                                                      | Date\_\_(UT)\_\_HR:MN
Astrometric RA & DEC         | 1          | degrees                | Astrometric RA and Dec with respect to the observer\'s location (GEOCETRIC)                                                | R.A.\_(ICRF)\_DEC
Observer sub-long& sub-lat   | 14         | degrees                | Apparent planetodetic (\"geodetic\") longitude and latitude of the center of the target seen by the OBSERVER at print-time | ob-lon, ob-lat
Solar sub-long & sub-lat     | 15         | degrees                | Apparent planetodetic (\"geodetic\") longitude and latitude of the Sun seen by the OBSERVER at print-time                  | Sl-lon, Sl-lat
North Pole Pos. ang. & dist. | 17         | degrees and arcseconds | Target\'s North Pole position angle and angular distance from the \"sub-observer\" point                                   | NP.ang, NP.ds
Helio range & range rate     | 19         | AU, km/s               | Heliocentric range (r) and range-rate (rdot)                                                                               | r, rdot
Observer range & range rate  | 20         | AU, km/s               | Range (delta) and range-rate (deldot) of the target center with respect to the observer                                    | delta, dedot
S-T-O angle                  | 24         | degrees                | Sun-Target-Observer angle                                                                                                  | S-T-O

The task **getephemtable** can be used to retrieve the ephemeris data from the JPL-Horizons system through its web service and convert a CASA table. (See also [Manipulate Ephemeris Objects page](ephemeris_data.ipynb#Manipulate-Ephemeris-Objects "Manipulate Ephemeris Objects")).


```
CASA <5>: getephemtable(objectname='Titan', timerange='2020/01/30~2020/01/31', interval='15m', outfile='Titan_test_ephem.tab')

```
The converted table contains following columns. Note that required columns by Measures, which is used to read the epehemeris table in CASA are MJD, RA, DEC, Rho, and RadVel. Other extra columns are added for use in **setjy** task.  

Column name | unit/format | description
--- | --- | ---
MJD | day | modified Julian date
RA | degree | atrometric right acension in ICRF/J2000 frame
DEC | degree | astrometric declination in ICRF/J2000 frame
Rho | AU | Geocentric distance
RadVal | AU/d | Geocentric distance rate
NP_ang | degree | North pole position angle
NP_dist | degree | North pole angular distance
DiskLong | degree | Sub-observer longitude
DiskLat | degree | Sub-observer latitude
Sl_lon | degree | Sub-Solar longitude
Sl_lat | degree | Sub-Solar latitude
r | AU | heliocentric distance
rdot | km/s | heliocentric distance rate
phang | degree | phase angle




### Array Configuration

Array configuration files for various telescopes are distributed with each CASA release. These configuration files can be used to define the telescope for simulator tools and tasks. Currently, configuration files for the following telescopes are available in CASA:

-   ALMA / 12m Array
-   ALMA / 7m ACA
-   VLA
-   VLBA
-   Next-Generation VLA (reference design)
-   ATCA
-   MeerKat
-   PdBI (pre-NOEMA)
-   WSRT
-   SMA
-   Carma

The full list of antenna configurations can be found in the [CASA Guides on Simulations](https://casaguides.nrao.edu/index.php?title=Antenna_Configurations_Models_in_CASA_Cycle6).

One can also locate the directory with the configurations in the CASA distribution and then list the configuration files, using the following commands in CASA:

```
CASA <1>: print(ctsys.resolve('alma/simmos/'))
/home/casa/data/distro/alma/simmos/

CASA <2>: ls /home/casa/data/distro/alma/simmos/
```

If a configuration file is not distributed with CASA but retrieved elsewhere, then the configuration file can be called by explicitly writing the full path to the location of the configuration file in the antennalist paramter of the simulator tasks.

<div class="alert alert-info">
**NOTE**: the most recent ALMA configuration files may not always be available in the most recent CASA version. ALMA configuration files for all cycles are available for download [here](https://almascience.nrao.edu/tools/casa-simulator). For the Next-Generation VLA reference design, the latest information can be found [here](https://ngvla.nrao.edu/page/tools).
</div>
